##  Part 1 simple linear regression
### To do

* Explain w/partner each line in this notebook and execute each cell  
* Explore relationship between quality of prediction and (size of training set / noise level)
    * Specifically discuss how you would automate this searching

### Discuss how you would implement the following calculations, with a function, as they are not available in sklearn (this is a big part of HW4!)

### Missing information for assessing coefficients: 

* 95% CI 
* Standard Error for $\beta_0$ and $\beta _1$
* P-value for test of $H_0$

### Missing information for assessing the model: 

* RSE , RSS , $R^2$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

from sklearn import linear_model
%matplotlib inline

In [ ]:
#model: Y = 3X + 4
#size of training data and scale of random noise 
pts=25 
noisescale=.25 
x=np.linspace(-50,50,num=pts)
B0=4
B1=3
yactual=B0+B1*x
np.random.seed(123) #for reproducible 
#add noise scaled to 25% of range to data
yrand=yactual+noisescale*(yactual.max()-yactual.min())*np.random.normal(size=pts)

In [ ]:
x

In [ ]:
yrand

In [ ]:
plt.plot(x,yactual,color='red',lw='3')
plt.scatter(x,yrand)

Compare x, x.reshape(pts,1), and x.reshape(-1,1)

In [ ]:
x.reshape(-1,1)

Doing SLR

In [ ]:
#EXPLAIN IN PLAIN LANGUAGE WHAT THIS LINE IS DOING!
regr=linear_model.LinearRegression() 
# note that x.reshape(-1,1) is required because I must strictly
# feed regr.fit a 200x1 array , np.linspace returned a list of numbers
regr.fit(x.reshape(-1,1),yrand.reshape(-1,1))
print('B0, B1: ',regr.intercept_, regr.coef_[0])

Compare your fitted functional with the actual fuctional

In [ ]:
plt.plot(x,yactual,color='red',lw='3',label='actual')
plt.plot(x,regr.predict(x.reshape(-1,1)),ls='--',label='fit')
plt.legend(loc='lower right')

### Take 10 mins and see if you can 

Useful plots to make after simple linear regression: 

1) X vs Y (showing both training data, fit model, and result of predictionsw on test data (if you have any) 

2) Y vs Y(hat):  this is called a *parity plot* 

3) X v s *residual* (Y - Y(hat)) 

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3))
ax.scatter(x, yactual, label='actual')
ax.scatter(x, yrand, label='measured')
ax.scatter(x, regr.predict(x.reshape(-1,1)), label='predicted')

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3))
ax.scatter(yrand, regr.predict(x.reshape(-1,1)), label='parity')

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3))
ax.scatter(x, regr.predict(x.reshape(-1, 1)) - yrand)
ax.plot(x, 0*x)

### Multiple linear regression

In this 2nd example, we generate model based on the idea that the $PCE$ of a candidate organic photovoltaic can be modeled as a contribution of the molecule's $mass$, $VOC$ and $E_{LUMO}$ values:  $PCE = \beta_0 + \beta_1*mass + \beta_2*VOC + \beta_3 * E_{LUMO}$

The extension from simple to multiple linear regression is trivial - can you figure it out with your partner or at your table?!

In [ ]:
harvard=pd.read_csv('https://raw.githubusercontent.com/UWDIRECT/UWDIRECT.github.io/master/Wi18_content/DSMCER/HCEPD_100K.csv')

Let's refresh our memory about harvard (it takes some time to do so)

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(harvard, alpha=0.2, figsize=(8, 8), diagonal='kde')

keep the same variable names - 

regr2=linear_model.LinearRegression()

regr2.fit(STUFF) 

Once your model is trained, can you make a parity plot: plot actual PCE data on x vs. trained PCE data on y. Also plot the line of parity so your eye can follow how good your predictions are doing....

### Assessing coefficients: 

In [ ]:
#model: Y = 3X + 4
#size of training data and scale of random noise 

In [ ]:
def regression(points=25, x_range=[-50,50], coefs=[4, 3], noisescale=0.25):
    pts=25 
    noisescale=.25 
    x=np.linspace(x_range[0],x_range[1],num=points)
    B0=coefs[0]
    B1=coefs[1]
    yactual=B0+B1*x
    #np.random.seed(123) #for reproducible 
    #add noise scaled to 25% of range to data
    yrand=yactual+noisescale*(yactual.max()-yactual.min())*np.random.normal(size=points)

    #EXPLAIN IN PLAIN LANGUAGE WHAT THIS LINE IS DOING!
    regr=linear_model.LinearRegression() 
    # note that x.reshape(-1,1) is required because I must strictly
    # feed regr.fit a 200x1 array , np.linspace returned a list of numbers
    regr.fit(x.reshape(-1,1),yrand.reshape(-1,1))
    return regr.intercept_[0], regr.coef_[0][0]

In [ ]:
repeats = 1000
B0 = [0]*repeats
B1 = [0]*repeats
for i in range(repeats):
    B0[i], B1[i] = regression(points=25, noisescale=0.25)

In [ ]:
repeats = 1000
noisescale = 0.25
B0 = [0]*repeats
B1 = [0]*repeats
for i in range(repeats):
    B0[i], B1[i] = regression(points=25, noisescale=noisescale)
print('95% CI Intercept: {} - {}'.format(np.round(np.percentile(np.array(B0), 5), 2), np.round(np.percentile(np.array(B0), 95), 2)))
print('95% CI Intercept: {} - {}'.format(np.round(np.percentile(np.array(B1), 5), 2), np.round(np.percentile(np.array(B1), 95), 2)))

In [ ]:
def regression_cis(points, noisescale, repeats):
    B0 = [0]*repeats
    B1 = [0]*repeats
    for i in range(repeats):
        B0[i], B1[i] = regression(points=points, noisescale=noisescale)
    print('95% CI Intercept: {} - {}'.format(np.round(np.percentile(np.array(B0), 5), 2), np.round(np.percentile(np.array(B0), 95), 2)))
    print('95% CI Slope: {} - {}'.format(np.round(np.percentile(np.array(B1), 5), 2), np.round(np.percentile(np.array(B1), 95), 2)))
    return [np.percentile(np.array(B0), 5), np.percentile(np.array(B0), 95)], [np.percentile(np.array(B1), 5), np.percentile(np.array(B1), 95)]

In [ ]:
B0cilo = []
B0cihi = []
B1cilo = []
B1cihi = []
points = [5, 10, 100, 1000, 10000, 100000]
for i in points:
    B0ci0, B1ci0 = regression_cis(i, noisescale, 1000)
    B0cilo.append(B0ci0[0])
    B0cihi.append(B0ci0[1])
    B1cilo.append(B1ci0[0])
    B1cihi.append(B1ci0[1])

fig, ax = plt.subplots(figsize=(5, 3))
ax.scatter(points, B0cilo, color='k')
ax.scatter(points, B0cihi, color='k')
plt.semilogx()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3))
ax.scatter(points, B1cilo, color='k')
ax.scatter(points, B1cihi, color='k')
plt.semilogx()

In [ ]:
B0cilo = []
B0cihi = []
B1cilo = []
B1cihi = []
points = [0.01, 0.05, 0.1, 0.25, 0.5, 1, 2, 5, 10, 20, 100, 200, 500, 1000]
for noise in points:
    B0ci0, B1ci0 = regression_cis(1000, noise, 1000)
    B0cilo.append(B0ci0[0])
    B0cihi.append(B0ci0[1])
    B1cilo.append(B1ci0[0])
    B1cihi.append(B1ci0[1])

fig, ax = plt.subplots(figsize=(5, 3))
ax.scatter(points, B0cilo, color='k')
ax.scatter(points, B0cihi, color='k')

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3))
ax.scatter(points, B1cilo, color='k')
ax.scatter(points, B1cihi, color='k')